In [1]:


import sys

from itertools import chain, combinations
from collections import defaultdict



def subsets(arr):
    """ Returns non empty subsets of arr"""
    return chain(*[combinations(arr, i + 1) for i, a in enumerate(arr)])


def returnItemsWithMinSupport(itemSet, transactionList, minSupport, freqSet):
    """calculates the support for items in the itemSet and returns a subset
    of the itemSet each of whose elements satisfies the minimum support"""
    _itemSet = set()
    localSet = defaultdict(int)

    for item in itemSet:
        for transaction in transactionList:
            if item.issubset(transaction):
                freqSet[item] += 1
                localSet[item] += 1
    # print("Frequency Set")
    # for key, value in freqSet.items():
    #     print(key,"->",value)

    for item, count in localSet.items():
        support = count

        if support >= minSupport:
            _itemSet.add(item)

    return _itemSet


def joinSet(itemSet, length):
    """Join a set with itself and returns the n-element itemsets"""
    return set(
        [i.union(j) for i in itemSet for j in itemSet if len(i.union(j)) == length]
    )


def getItemSetTransactionList(data_iterator):
    transactionList = list()
    itemSet = set()
    for record in data_iterator:
       
        transaction = record
        
        transactionList.append(transaction)
        for item in transaction:
            itemSet.add(frozenset([item]))  # Generate 1-itemSets
    # print(transactionList)  set of record frozen list
    # print(itemSet)
    return itemSet, transactionList

def intermediate_results(table,index_val,flg):
    L_list=[]
    C_list=[]

    if flg==0:
        
        for index,val in enumerate(table):

            C_list.append(list(val))
        print()
        print("C{} Table".format(index_val))
        print(C_list)

    else:
        for index,val in enumerate(table):

            L_list.append(list(val))

        print()
        print("L{} Table".format(index_val))
        print(L_list)



    
    

def runApriori(data_iter, minSupport, minConfidence):
    """
    run the apriori algorithm. data_iter is a record iterator
    Return both:
     - items (tuple, support)
     - rules ((pretuple, posttuple), confidence)
    """
    itemSet, transactionList = getItemSetTransactionList(data_iter)
    

    freqSet = defaultdict(int)
    largeSet = dict()
    # Global dictionary which stores (key=n-itemSets,value=support)
    # which satisfy minSupport

    assocRules = dict()
    # Dictionary which stores Association Rules

    oneCSet = returnItemsWithMinSupport(itemSet, transactionList, minSupport, freqSet)
    C1_list=[]
    L1_list=[]
    for index,val in enumerate(oneCSet):

        C1_list.append(list(val))
    print()
    print("*********  Intermediate Results  *************")
    print()
    print("C1 Table")
    print(C1_list)

    currentLSet = oneCSet
    for index,val in enumerate(currentLSet):

        L1_list.append(list(val))

    print()
    print("L1 Table")
    print(L1_list)

    k = 2
    while currentLSet != set([]):
        print()
        print("***********************************************")
        largeSet[k - 1] = currentLSet
        currentLSet = joinSet(currentLSet, k)
        intermediate_results(currentLSet,k,0)
        currentCSet = returnItemsWithMinSupport(
            currentLSet, transactionList, minSupport, freqSet
        )
       
        currentLSet = currentCSet
        intermediate_results(currentCSet,k,1)
        k = k + 1

    def getSupport(item):
        """local function which Returns the support of an item"""
        return freqSet[item]

    toRetItems = []
    for key, value in largeSet.items():
        toRetItems.extend([(tuple(item), getSupport(item)) for item in value])

    toRetRules = []
    for key, value in list(largeSet.items())[1:]:
        for item in value:
            _subsets = map(frozenset, [x for x in subsets(item)])
            for element in _subsets:
                remain = item.difference(element)
                if len(remain) > 0:
                    confidence = getSupport(item) / getSupport(element)
                    if confidence >= minConfidence:
                        toRetRules.append(((tuple(element), tuple(remain)), confidence))
    return toRetItems, toRetRules


def printResults(items, rules):
    """prints the generated itemsets sorted by support and the confidence rules sorted by confidence"""
    print()
    print("***** Items, Support is described as ************")
    print()
    for item, support in sorted(items, key=lambda x: x[1]):
        print("item: %s , %d" % (str(item), support))
    print()
    print("\nAssociative Rules of length 3 is:")
    print()
    
    for rule, confidence in sorted(rules, key=lambda x: x[1]):
        pre, post = rule
        if len(pre)==2 or len(post)==2:
            print("Rule: %s ==> %s , %d" % (str(pre), str(post), confidence*100))


def to_str_results(items, rules):
    """prints the generated itemsets sorted by support and the confidence rules sorted by confidence"""
    i, r = [], []
    for item, support in sorted(items, key=lambda x: x[1]):
        x = "item: %s , %.3f" % (str(item), support)
        i.append(x)

    for rule, confidence in sorted(rules, key=lambda x: x[1]):
        pre, post = rule
        x = "Rule: %s ==> %s , %.3f" % (str(pre), str(post), confidence)
        r.append(x)

    return i, r


def dataFromFile(fname):
    
    """Function which reads from the file and yields a generator"""
    with open(fname, "r") as file_iter:
        for line in file_iter:
            
            line = line.strip().rstrip(",")  # Remove trailing comma
            # print(line)
            record = frozenset(line.split(","))
            
            yield record


if __name__ == "__main__":

    minSupport = int(input("Enter the Minimum Support:"))
    minConfidence = float(input("Enter the Min Confidence:"))
    # minSupport=2
    # minConfidence=0.7
    dataset_name="AllElectronics.csv"
    inFile = dataFromFile(dataset_name)

   

    items, rules = runApriori(inFile, minSupport, minConfidence)

    printResults(items, rules)

Enter the Minimum Support:2
Enter the Min Confidence:0.7

*********  Intermediate Results  *************

C1 Table
[['I5'], ['I3'], ['I1'], ['I4'], ['I2']]

L1 Table
[['I5'], ['I3'], ['I1'], ['I4'], ['I2']]

***********************************************

C2 Table
[['I5', 'I1'], ['I1', 'I4'], ['I4', 'I3'], ['I5', 'I2'], ['I4', 'I2'], ['I1', 'I3'], ['I2', 'I3'], ['I1', 'I2'], ['I5', 'I4'], ['I5', 'I3']]

L2 Table
[['I5', 'I1'], ['I5', 'I2'], ['I4', 'I2'], ['I1', 'I3'], ['I2', 'I3'], ['I1', 'I2']]

***********************************************

C3 Table
[['I5', 'I4', 'I2'], ['I1', 'I2', 'I3'], ['I4', 'I2', 'I3'], ['I1', 'I4', 'I2'], ['I5', 'I1', 'I3'], ['I5', 'I1', 'I2'], ['I5', 'I2', 'I3']]

L3 Table
[['I1', 'I2', 'I3'], ['I5', 'I1', 'I2']]

***********************************************

C4 Table
[['I1', 'I2', 'I3', 'I5']]

L4 Table
[]

***** Items, Support is described as ************

item: ('I5',) , 2
item: ('I4',) , 2
item: ('I5', 'I1') , 2
item: ('I5', 'I2') , 2
item: ('I4', '